In [1]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
path_root = '/home/dmr/DatasetForExperiments/scareware/'

In [2]:
from keras.preprocessing.image import ImageDataGenerator
batches = ImageDataGenerator().flow_from_directory(directory=path_root, target_size=(64,64), batch_size=10000)

2023-05-03 16:03:08.422795: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Found 97 images belonging to 11 classes.


In [3]:
# Print information about Images 
imgs, labels = next(batches)
imgs.shape # Images with width x length x depth
labels.shape # Lablels with batch_size, number of classes

(97, 11)

In [4]:
import numpy as np
import scipy as sp
from PIL import Image

# Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs/255.,labels, test_size=0.3)

In [5]:
X_train.shape

(67, 64, 64, 3)

In [6]:
X_test.shape

(30, 64, 64, 3)

In [7]:
y_train.shape

(67, 11)

In [8]:
y_test.shape

(30, 11)

In [9]:
# CNN Model

In [10]:
import keras
import tensorflow
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import AUC

In [11]:
num_classes = 11 # Changed from 10 to 11 for families on scareware

In [12]:
def malware_model():
    Malware_model = Sequential()
    # Convolutional Layer : 30 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(30, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))

    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #Convolutional Layer : 15 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(18, (3, 3), activation='relu'))
    
    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # DropOut Layer : Dropping 25% of neurons.
    Malware_model.add(Dropout(0.25))
    
    # Flatten Layer
    Malware_model.add(Flatten())
    
    # Dense/Fully Connected Layer : 128 Neurons, Relu activation function
    Malware_model.add(Dense(128, activation='relu'))    
   # Malware_model.add(Dense(64, activation='relu'))
    
    # DropOut Layer : Dropping 50% of neurons.
    Malware_model.add(Dropout(0.5))
    
    # Dense/Fully Connected Layer : 50 Neurons, Softmax activation function
    Malware_model.add(Dense(50, activation='relu'))
    
    # Dense/Fully Connected Layer : num_class Neurons, Softmax activation function
    Malware_model.add(Dense(num_classes, activation='softmax'))
    
    
    Malware_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return Malware_model


    # sparse_categorical_cross_entropy
    # categorical_crossentropy
    

In [13]:
from keras import backend as K

def recall_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_test, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_test, y_pred):
    precision = precision_m(y_test, y_pred)
    recall = recall_m(y_test, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
Malware_model = malware_model()

2023-05-03 16:03:11.536100: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-05-03 16:03:11.858655: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-03 16:03:11.858933: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-05-03 16:03:11.860903: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 16:03:11.880025: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz
2023-05-03 16:03:11.8815

In [15]:
Malware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 30)        840       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 18)        4878      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 18)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 18)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3528)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [16]:
y_train.shape

(67, 11)

In [17]:
y_train_new = np.argmax(y_train, axis=1)

In [18]:
y_train_new

array([10, 10,  1, 10,  5, 10,  6,  8, 10,  3,  2,  3,  2, 10,  1,  1,  8,
        6,  4,  6,  4,  1,  8, 10,  9,  7, 10,  4,  6,  7,  3,  4,  3,  9,
        5,  4,  3,  6,  9,  4,  5, 10,  5,  9,  1, 10,  7,  9,  7,  6,  0,
        0,  2,  9,  1,  6,  7,  5,  8,  1,  2,  1,  5,  5,  3,  8,  2])

In [19]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(y_train_new),
                                                 y = y_train_new)

class_weights = {l:c for l,c in zip(np.unique(y_train_new), class_weights)}

In [20]:
Malware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m,precision_m, recall_m,AUC()])

In [21]:
Malware_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,  class_weight=class_weights)

2023-05-03 16:03:12.055843: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/100
3/3 [==============================] - 1s 257ms/step - loss: 0.6061 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.4896 - val_loss: 0.3232 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5733
Epoch 2/100
3/3 [==============================] - 0s 37ms/step - loss: 0.3877 - f1_m: 0.0556 - precision_m: 0.1548 - recall_m: 0.0339 - auc: 0.4775 - val_loss: 0.3422 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5701
Epoch 3/100
3/3 [==============================] - 0s 39ms/step - loss: 0.3781 - f1_m: 0.0516 - precision_m: 0.1083 - recall_m: 0.0339 - auc: 0.5276 - val_loss: 0.3067 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5719
Epoch 4/100
3/3 [==============================] - 0s 39ms/step - loss: 0.3286 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.5621 - val_loss: 0.3180 - val_f1_

3/3 [==============================] - 0s 40ms/step - loss: 0.2174 - f1_m: 0.2684 - precision_m: 0.7656 - recall_m: 0.1641 - auc: 0.9172 - val_loss: 0.2385 - val_f1_m: 0.2353 - val_precision_m: 1.0000 - val_recall_m: 0.1333 - val_auc: 0.8300
Epoch 32/100
3/3 [==============================] - 0s 37ms/step - loss: 0.2161 - f1_m: 0.3434 - precision_m: 0.9583 - recall_m: 0.2335 - auc: 0.8833 - val_loss: 0.2414 - val_f1_m: 0.3684 - val_precision_m: 0.8750 - val_recall_m: 0.2333 - val_auc: 0.8235
Epoch 33/100
3/3 [==============================] - 0s 38ms/step - loss: 0.2252 - f1_m: 0.3472 - precision_m: 0.8351 - recall_m: 0.2209 - auc: 0.8725 - val_loss: 0.2410 - val_f1_m: 0.4500 - val_precision_m: 0.9000 - val_recall_m: 0.3000 - val_auc: 0.8191
Epoch 34/100
3/3 [==============================] - 0s 38ms/step - loss: 0.2111 - f1_m: 0.5254 - precision_m: 0.8351 - recall_m: 0.3924 - auc: 0.9248 - val_loss: 0.2317 - val_f1_m: 0.4500 - val_precision_m: 0.9000 - val_recall_m: 0.3000 - val_auc: 

3/3 [==============================] - 0s 40ms/step - loss: 0.0802 - f1_m: 0.8493 - precision_m: 0.9441 - recall_m: 0.7760 - auc: 0.9949 - val_loss: 0.2160 - val_f1_m: 0.5306 - val_precision_m: 0.6842 - val_recall_m: 0.4333 - val_auc: 0.8611
Epoch 64/100
3/3 [==============================] - 0s 39ms/step - loss: 0.0558 - f1_m: 0.9413 - precision_m: 0.9813 - recall_m: 0.9049 - auc: 0.9973 - val_loss: 0.2169 - val_f1_m: 0.6122 - val_precision_m: 0.7895 - val_recall_m: 0.5000 - val_auc: 0.8696
Epoch 65/100
3/3 [==============================] - 0s 38ms/step - loss: 0.0858 - f1_m: 0.8601 - precision_m: 0.9190 - recall_m: 0.8099 - auc: 0.9938 - val_loss: 0.2179 - val_f1_m: 0.5833 - val_precision_m: 0.7778 - val_recall_m: 0.4667 - val_auc: 0.8629
Epoch 66/100
3/3 [==============================] - 0s 41ms/step - loss: 0.0680 - f1_m: 0.8528 - precision_m: 0.9058 - recall_m: 0.8064 - auc: 0.9956 - val_loss: 0.2191 - val_f1_m: 0.6000 - val_precision_m: 0.7500 - val_recall_m: 0.5000 - val_auc: 

3/3 [==============================] - 0s 52ms/step - loss: 0.0354 - f1_m: 0.9431 - precision_m: 0.9556 - recall_m: 0.9310 - auc: 0.9988 - val_loss: 0.2844 - val_f1_m: 0.6667 - val_precision_m: 0.7037 - val_recall_m: 0.6333 - val_auc: 0.8488
Epoch 96/100
3/3 [==============================] - 0s 42ms/step - loss: 0.0185 - f1_m: 0.9914 - precision_m: 1.0000 - recall_m: 0.9831 - auc: 1.0000 - val_loss: 0.2852 - val_f1_m: 0.6441 - val_precision_m: 0.6552 - val_recall_m: 0.6333 - val_auc: 0.8335
Epoch 97/100
3/3 [==============================] - 0s 47ms/step - loss: 0.0145 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - auc: 1.0000 - val_loss: 0.3170 - val_f1_m: 0.5714 - val_precision_m: 0.6154 - val_recall_m: 0.5333 - val_auc: 0.8312
Epoch 98/100
3/3 [==============================] - 0s 40ms/step - loss: 0.0305 - f1_m: 0.9398 - precision_m: 0.9444 - recall_m: 0.9353 - auc: 0.9999 - val_loss: 0.3204 - val_f1_m: 0.6552 - val_precision_m: 0.6786 - val_recall_m: 0.6333 - val_auc: 

In [22]:
scores = Malware_model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 25ms/step - loss: 0.2484 - f1_m: 0.6792 - precision_m: 0.7826 - recall_m: 0.6000 - auc: 0.8704


In [23]:
loss, accuracy, f1_score, precision, recall = Malware_model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 21ms/step - loss: 0.2484 - f1_m: 0.6792 - precision_m: 0.7826 - recall_m: 0.6000 - auc: 0.8704


In [24]:
print('Final CNN accuracy: ', scores[1])

Final CNN accuracy:  0.6792452335357666
